# Getting started with WYM

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/softlab-unimore/WYM/blob/master/quick_start_wym.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly explaining entity matching. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train a model to explain using supervised learning.

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [1]:
!pip install --upgrade --no-cache-dir gdown
!gdown --id 14eQiG9sYM4LcCfJbwnoUgxZ0-PfpsWkQ -O ABT-BUY-data.zip
!unzip ABT-BUY-data.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=14eQiG9sYM4LcCfJbwnoUgxZ0-PfpsWkQ
To: /content/ABT-BUY-data.zip
100% 39.6M/39.6M [00:00<00:00, 51.9MB/s]
Archive:  ABT-BUY-data.zip
   creating: Abt-Buy/
  inflating: Abt-Buy/train_simplified.csv  
  inflating: Abt-Buy/test_simplified.csv  
   creating: Abt-Buy/.ipynb_checkpoints/
  inflating: Abt-Buy/train_merged.csv  
  inflating: Abt-Buy/valid_simplified.csv  
  inflating: Abt-Buy/valid_merged.csv  
  inflating: Abt-Buy/test_merged.csv  
  inflating: Abt-Buy/best_model.pth  
  i

## Clone WYM

In [ ]:
!git clone https://github.com/softlab-unimore/WYM.git
!pip install -r wym/requirements.txt